In [1]:
import argparse

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import Optimizer, nihgcn
from myutils import *
from sampler import BalancedSampler
from sklearn.model_selection import KFold

In [3]:
class Args:
    def __init__(self):
        self.device = "cpu"  # cuda:number or cpu
        self.data = "gdsc1"  # Dataset{gdsc or ccle}
        self.lr = 0.001  # the learning rate
        self.wd = 1e-5  # the weight decay for l2 normalizaton
        self.layer_size = [1024, 1024]  # Output sizes of every layer
        self.alpha = 0.25  # the scale for balance gcn and ni
        self.gamma = 8  # the scale for sigmod
        self.epochs = 1000  # the epochs for model


args = Args()

In [4]:
res, drug_finger, exprs, null_mask = load_data(args)

load gdsc1


In [5]:
adj_coo = sp.coo_matrix(res)
pos_edges = np.vstack((adj_coo.row, adj_coo.col)).T

# ネガティブエッジ候補作成
adj_dense = adj_coo.toarray()

# ポジティブ（接続ありかつ null_mask==0）
pos_mask = np.logical_and(adj_dense == 1, null_mask == 0)
pos_row, pos_col = np.where(pos_mask)
pos_edges = np.vstack((pos_row, pos_col)).T
pos_labels = np.ones(len(pos_edges), dtype=int)

# ネガティブ（接続なしかつ null_mask==0）
neg_mask = np.logical_and(adj_dense == 0, null_mask == 0)
neg_row, neg_col = np.where(neg_mask)
neg_edges = np.vstack((neg_row, neg_col)).T
neg_labels = np.zeros(len(neg_edges), dtype=int)

# 統合
all_edges = np.vstack((pos_edges, neg_edges))
all_labels = np.concatenate((pos_labels, neg_labels))

In [6]:
k = 5
kf = KFold(n_splits=5, shuffle=True, random_state=42)

true_datas = pd.DataFrame()
predict_datas = pd.DataFrame()

for i, (train_idx, test_idx) in enumerate(kf.split(all_edges)):
    train_edges = all_edges[train_idx]
    train_labels = all_labels[train_idx]
    test_edges = all_edges[test_idx]
    test_labels = all_labels[test_idx]

    sampler = BalancedSampler(
        edge_train=train_edges,
        label_train=train_labels,
        edge_test=test_edges,
        label_test=test_labels,
        adj_shape=res.shape,
        seed=i
    )
    
    model = nihgcn(
        adj_mat=sampler.train_data,
        cell_exprs=exprs,
        drug_finger=drug_finger,
        layer_size=args.layer_size,
        alpha=args.alpha,
        gamma=args.gamma,
        device=args.device,
    ).to(args.device)
    
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=args.lr,
        wd=args.wd,
        epochs=args.epochs,
        device=args.device,
    ).to(args.device)
    true_data, predict_data = opt()
    true_datas = pd.concat([true_datas, translate_result(true_data)], ignore_index=True)
    predict_datas = pd.concat(
        [predict_datas, translate_result(predict_data)], ignore_index=True
    )

epoch:   0 loss:0.705250 auc:0.4876
epoch:  20 loss:0.345802 auc:0.8670
epoch:  40 loss:0.233868 auc:0.8951
epoch:  60 loss:0.172651 auc:0.9057
epoch:  80 loss:0.140148 auc:0.9111
epoch: 100 loss:0.122395 auc:0.9133
epoch: 120 loss:0.112959 auc:0.9120
epoch: 140 loss:0.104348 auc:0.9126
epoch: 160 loss:0.099046 auc:0.9116
epoch: 180 loss:0.095300 auc:0.9109
epoch: 200 loss:0.092917 auc:0.9105
epoch: 220 loss:0.090682 auc:0.9091
epoch: 240 loss:0.090770 auc:0.9089
epoch: 260 loss:0.087999 auc:0.9079
epoch: 280 loss:0.087092 auc:0.9074
epoch: 300 loss:0.087242 auc:0.9081
epoch: 320 loss:0.085612 auc:0.9069
epoch: 340 loss:0.084901 auc:0.9065
epoch: 360 loss:0.086238 auc:0.9056
epoch: 380 loss:0.084262 auc:0.9064
epoch: 400 loss:0.083762 auc:0.9060
epoch: 420 loss:0.093452 auc:0.9037
epoch: 440 loss:0.084488 auc:0.9055
epoch: 460 loss:0.083235 auc:0.9057
epoch: 480 loss:0.082890 auc:0.9053
epoch: 500 loss:0.085863 auc:0.9033
epoch: 520 loss:0.083784 auc:0.9054
epoch: 540 loss:0.082631 auc

In [7]:
true_datas.to_csv(f"true_{args.data}.csv")
predict_datas.to_csv(f"pred_{args.data}.csv")